In [4]:
import cv2
import numpy as np
import glob
import os
from PIL import Image


def load_rgb_images(path):
    image_paths = [
        os.path.join(path, f)
        for f in os.listdir(path) 
        if f.endswith('.tiff')
    ]
    exposures = []
    images = []
    for p in sorted(image_paths):
        img = Image.open(p).convert('RGB')
        img_np = np.array(img).astype(np.float32)
        images.append(img_np)
        # If the filename is "0.1.tiff", then exposure_time = 0.1
        filename = os.path.splitext(os.path.basename(p))[0]
        exposure_time = 1/ float(filename)
        exposures.append(exposure_time)
    images = np.stack(images, axis=0)  # shape (P, H, W, 3)
    exposures = np.array(exposures)    # shape (P,)
    return images, exposures


In [6]:
data_path = "../data/raw/"
output_path = "../data/output/robertson/"
# 1. Load images
image_files = sorted(glob.glob("../data/raw/*.tiff"))
ldr_images = [cv2.imread(img).astype(np.uint8) for img in image_files]

# 2. Specify exposure times (in seconds, float32)
exposure_times = np.array([1/30.0, 0.25, 2.5, 15.0], dtype=np.float32)  # Replace with your real values

# 3. Calibrate camera response using Robertson method
calibrate = cv2.createCalibrateRobertson()
response = calibrate.process(ldr_images, exposure_times)

# 4. Merge images into HDR using the same method (Robertson)
merge = cv2.createMergeRobertson()
hdr = merge.process(ldr_images, exposure_times, response)

# 5. Tonemap for visualization (e.g., using Reinhard tonemapping)
tonemap = cv2.createTonemapReinhard(gamma=1.5)
ldr_tonemapped = tonemap.process(hdr)

# 6. Convert to 8-bit and save
ldr_8bit = np.clip(ldr_tonemapped * 255, 0, 255).astype('uint8')
cv2.imwrite("ldr_tonemapped.jpg", ldr_8bit)


/tmp/ipykernel_1712453/1570459403.py:23: RuntimeWarning: invalid value encountered in cast
  ldr_8bit = np.clip(ldr_tonemapped * 255, 0, 255).astype('uint8')


True